# **LAB SESSION 2 - Bagging and Random Forests**

## Utils

*Note: in all this project, use **the value 0** when you need to choose a value for a random state*

## Ex 1: comparison between random forests and bagging

We work with the [Urban Land Cover data base](https://archive.ics.uci.edu/ml/datasets/Urban+Land+Cover). The data are used for automated mapping of urban land cover (trees, grass, soil, concrete, asphalt, buildings, etc.) in satellite or aerial imagery. Nine types of urban land cover are considered and multi-scale spectral, size, shape, and texture information are used for classification. The data consists in a train set and a test set. **The goal is to predict the urban land cover (the variable named `class`) based on the multi-scale spectral, size, shape, and texture information. It is then a classification problem. We will use the overall accuracy (1-misclassification rate) as performance criterion**. Note that other preformnce criterion exist for classification problem such as specificity, sensitivity, F-score, etc.

Before to start, we: 
 1) load the data and perform a briel descriptive analysis of them;
 2) select the variables that we will use in the exercise, 


In [35]:
# Load the data 
import pandas as pd # data analysis
ulc_train = pd.read_csv("ULC_training.csv") 
ulc_test = pd.read_csv("ULC_testing.csv") 

In [36]:
# Display the dimension
print(ulc_train.shape)
print(ulc_test.shape)

(168, 148)
(507, 148)


In [37]:
# Have a first quick look at the datasets and display the dimension
ulc_train.head()

,class,BrdIndx,Area,Round,Bright,Compact,ShpIndx,Mean_G,Mean_R,Mean_NIR,...,SD_NIR_140,LW_140,GLCM1_140,Rect_140,GLCM2_140,Dens_140,Assym_140,NDVI_140,BordLngth_140,GLCM3_140
0,car,1.27,91,0.97,231.38,1.39,1.47,207.92,241.74,244.48,...,26.18,2.00,0.50,0.85,6.29,1.67,0.70,-0.08,56,3806.36
1,concrete,2.36,241,1.56,216.15,2.46,2.51,187.85,229.39,231.20,...,22.29,2.25,0.79,0.55,8.42,1.38,0.81,-0.09,1746,1450.14
2,concrete,2.12,266,1.47,232.18,2.07,2.21,206.54,244.22,245.79,...,15.59,2.19,0.76,0.74,7.24,1.68,0.81,-0.07,566,1094.04
3,concrete,2.42,399,1.28,230.40,2.49,2.73,204.60,243.27,243.32,...,13.51,3.34,0.82,0.74,7.44,1.36,0.92,-0.09,1178,1125.38
4,concrete,2.15,944,1.73,193.18,2.28,4.10,165.98,205.55,208.00,...,15.65,50.08,0.85,0.49,8.15,0.23,1.00,-0.08,6232,1146.38


In [38]:
ulc_test.head()

,class,BrdIndx,Area,Round,Bright,Compact,ShpIndx,Mean_G,Mean_R,Mean_NIR,...,SD_NIR_140,LW_140,GLCM1_140,Rect_140,GLCM2_140,Dens_140,Assym_140,NDVI_140,BordLngth_140,GLCM3_140
0,concrete,1.32,131,0.81,222.74,1.66,2.18,192.94,235.11,240.15,...,31.15,5.04,0.80,0.58,8.56,0.82,0.98,-0.10,1512,1287.52
1,shadow,1.59,864,0.94,47.56,1.41,1.87,36.82,48.78,57.09,...,12.01,3.70,0.52,0.96,7.01,1.69,0.86,-0.14,196,2659.74
2,shadow,1.41,409,1.00,51.38,1.37,1.53,41.72,51.96,60.48,...,18.75,3.09,0.90,0.63,8.32,1.38,0.84,0.10,1198,720.38
3,tree,2.58,187,1.91,70.08,3.41,3.11,93.13,55.20,61.92,...,27.67,6.33,0.89,0.70,8.56,1.10,0.96,0.20,524,891.36
4,asphalt,2.60,116,2.05,89.57,3.06,3.02,73.17,94.89,100.64,...,32.05,1.01,0.83,0.75,8.62,2.08,0.08,-0.10,496,1194.76


In [39]:
## Display a statistic summary for the numerical variables
ulc_train.describe(include='all')

,class,BrdIndx,Area,Round,Bright,Compact,ShpIndx,Mean_G,Mean_R,Mean_NIR,...,SD_NIR_140,LW_140,GLCM1_140,Rect_140,GLCM2_140,Dens_140,Assym_140,NDVI_140,BordLngth_140,GLCM3_140
count,168,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,...,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000,168.000000
unique,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,grass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2.008512,565.869048,1.132976,165.569821,2.077679,2.229881,161.577083,163.672440,171.459226,...,23.769881,3.098274,0.796488,0.665000,7.795536,1.594405,0.615357,0.014583,983.309524,1275.292917
std,NaN,0.634807,679.852886,0.489150,61.883993,0.699600,0.703572,63.407201,71.306748,67.973969,...,12.836522,6.101883,0.103930,0.179086,0.670491,0.460627,0.239900,0.153677,880.013745,603.658611
min,NaN,1.000000,10.000000,0.020000,37.670000,1.000000,1.060000,30.680000,32.210000,40.120000,...,4.020000,1.000000,0.330000,0.240000,6.290000,0.230000,0.070000,-0.360000,56.000000,336.730000
25%,NaN,1.537500,178.000000,0.787500,133.977500,1.547500,1.700000,91.040000,101.187500,120.165000,...,13.965000,1.395000,0.757500,0.560000,7.357500,1.325000,0.460000,-0.080000,320.000000,817.405000
50%,NaN,1.920000,315.000000,1.085000,164.485000,1.940000,2.130000,187.560000,160.615000,178.345000,...,21.135000,1.740000,0.810000,0.690000,7.790000,1.660000,0.620000,-0.040000,776.000000,1187.025000
75%,NaN,2.375000,667.000000,1.410000,221.895000,2.460000,2.680000,210.940000,234.815000,236.002500,...,29.957500,2.285000,0.870000,0.810000,8.260000,1.945000,0.810000,0.120000,1412.500000,1588.427500


In [40]:
ulc_test.describe(include='all')

,class,BrdIndx,Area,Round,Bright,Compact,ShpIndx,Mean_G,Mean_R,Mean_NIR,...,SD_NIR_140,LW_140,GLCM1_140,Rect_140,GLCM2_140,Dens_140,Assym_140,NDVI_140,BordLngth_140,GLCM3_140
count,507,507.000000,507.000000,507.000000,507.000000,507.000000,507.000000,507.000000,507.000000,507.000000,...,507.000000,507.000000,507.000000,507.000000,507.000000,507.000000,507.000000,507.000000,507.000000,507.000000
unique,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,building,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2.025720,562.504931,1.237574,165.612939,2.187081,2.277318,166.290355,162.291953,168.256667,...,24.601144,2.931657,0.817712,0.597732,8.048698,1.455838,0.653905,0.027436,1398.706114,1101.998185
std,NaN,0.619254,698.655240,0.561988,63.230806,0.874054,0.718441,59.217648,73.455101,69.702475,...,12.203441,4.942887,0.106007,0.197505,0.787912,0.451781,0.251287,0.133834,1097.323462,533.927869
min,NaN,1.000000,22.000000,0.000000,26.850000,1.000000,1.040000,22.910000,26.520000,31.110000,...,2.650000,1.000000,0.200000,0.100000,5.690000,0.240000,0.030000,-0.360000,34.000000,211.270000
25%,NaN,1.580000,159.000000,0.840000,127.485000,1.650000,1.715000,146.460000,97.585000,111.715000,...,14.485000,1.375000,0.770000,0.455000,7.370000,1.160000,0.470000,-0.080000,601.000000,726.745000
50%,NaN,1.950000,323.000000,1.210000,170.650000,2.000000,2.180000,189.630000,158.280000,167.750000,...,22.380000,1.920000,0.840000,0.610000,8.020000,1.440000,0.710000,-0.020000,1148.000000,1011.230000
75%,NaN,2.380000,681.500000,1.565000,224.825000,2.490000,2.675000,206.780000,237.375000,238.480000,...,33.825000,2.800000,0.890000,0.760000,8.750000,1.775000,0.860000,0.145000,1874.000000,1335.640000


In [41]:
## Display the distribution of the target variable in the two datasets
ulc_train['class'].value_counts()


grass        29
building     25
concrete     23
tree         17
shadow       16
car          15
pool         15
asphalt      14
soil         14
Name: class, dtype: int64

In [42]:
ulc_test['class'].value_counts()

building     97
concrete     93
tree         89
grass        83
shadow       45
asphalt      45
car          21
soil         20
pool         14
Name: class, dtype: int64

In [43]:
# Select the variables that we be used 
target_column = "class" # The response variable that we will consider
features_columns = list(ulc_train)
features_columns.remove('class') # The predictors/features used to predict the target
print(target_column)
print(features_columns)

class
['BrdIndx', 'Area', 'Round', 'Bright', 'Compact', 'ShpIndx', 'Mean_G', 'Mean_R', 'Mean_NIR', 'SD_G', 'SD_R', 'SD_NIR', 'LW', 'GLCM1', 'Rect', 'GLCM2', 'Dens', 'Assym', 'NDVI', 'BordLngth', 'GLCM3', 'BrdIndx_40', 'Area_40', 'Round_40', 'Bright_40', 'Compact_40', 'ShpIndx_40', 'Mean_G_40', 'Mean_R_40', 'Mean_NIR_40', 'SD_G_40', 'SD_R_40', 'SD_NIR_40', 'LW_40', 'GLCM1_40', 'Rect_40', 'GLCM2_40', 'Dens_40', 'Assym_40', 'NDVI_40', 'BordLngth_40', 'GLCM3_40', 'BrdIndx_60', 'Area_60', 'Round_60', 'Bright_60', 'Compact_60', 'ShpIndx_60', 'Mean_G_60', 'Mean_R_60', 'Mean_NIR_60', 'SD_G_60', 'SD_R_60', 'SD_NIR_60', 'LW_60', 'GLCM1_60', 'Rect_60', 'GLCM2_60', 'Dens_60', 'Assym_60', 'NDVI_60', 'BordLngth_60', 'GLCM3_60', 'BrdIndx_80', 'Area_80', 'Round_80', 'Bright_80', 'Compact_80', 'ShpIndx_80', 'Mean_G_80', 'Mean_R_80', 'Mean_NIR_80', 'SD_G_80', 'SD_R_80', 'SD_NIR_80', 'LW_80', 'GLCM1_80', 'Rect_80', 'GLCM2_80', 'Dens_80', 'Assym_80', 'NDVI_80', 'BordLngth_80', 'GLCM3_80', 'BrdIndx_100', '

In [44]:
data, target = ulc_train[features_columns], ulc_train[target_column]
data_test, target_test = ulc_test[features_columns], ulc_test[target_column]

Question 1. Fit a random forest named *`rfc`* on the train set to explain the type of urban land cover (variable `class`) according to multi-scale spectral, size, shape, and texture information. More specifically, you will use the [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier) class. Read carefully the documentation.

*Indication: for the hyperparameters, use the values `n_estimators = 500` and `max_features= sqrt(d)` with `d` denoting the number of features, `oob_score=True`  and `random_state=0`.*




Before buiding `rfc` remind in the window below the meaning of`n_estimators` and `max_features`. 

Your answer:.............

In [33]:
n_features=sqrt(data.shape[1])
print(n_features)

12.12435565298214


In [45]:
##------- Complete the command below by filling in the gaps '...'.-------##

# Step 1: create the object rfc, it is a RandomForestClassifier object with n_estimators=500, max_features='auto' and random_state=0
from math import sqrt
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=500,max_features="auto",oob_score=True,random_state=0)

# Step 2: build the random forest on the train set by indicating the input data and the target variable 
rfc.fit(data,target)

RandomForestClassifier(max_features=12, n_estimators=500, oob_score=True,
                       random_state=0)

In [ ]:
## Solution

# Step 1: create the object rfc, it is a RandomForestClassifier object with n_estimators=500, max_features='auto' and random_state=0
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=500, max_features=12,oob_score=True,random_state=0)

# Step 2: build the random forest on the train set by indicating the input data and the target variable 
rfc.fit(data, target)

In [46]:
##------- Complete the command below by filling in the gaps '...'.-------##

# Step 3: look at the parameters used by your forest
from pprint import pprint
print('Parameters of the forest:\n')
pprint(rfc.get_params())
print('\n')

Parameters of the forest:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 12,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 500,
 'n_jobs': None,
 'oob_score': True,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}




Compute the oob error and comment the result. What does it represent ?

In [47]:
##------- Complete the command below by filling in the gaps '...'.-------##
# Step 4: print the oob_score (attributes of rfc named oob_score_)
pprint(rfc.get_params('oob_score'))

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 12,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 500,
 'n_jobs': None,
 'oob_score': True,
 'random_state': 0,
 'verbose': 0,
 'warm_start': False}


In [ ]:
# Step 4: print the oob_score (attributes of rfc named oob_score_)
print('OOB error:'),
print(rfc.oob_score_)

Meaning of the OOB error and result interpretation:

Comment: .............

Question 2. Predict the class of each observation of the test sample by using the random forest `rfc`and display the confusion matrix. Comment it. How many observations are misclassified ? Compute the accuracy.

In [ ]:
##------- Complete the command below by filling in the gaps '...'.-------##

# Step 1: compute the predictions on the test set
predictions_test = rfc.predict(.......) 

# Step 2: display the confusion matrix on the test set

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(....., .......)
disp = ConfusionMatrixDisplay(confusion_matrix=.....,display_labels=rfc.classes_)
disp.plot() 

# Step 3: compute the accuracy on the test set
accuracy = rfc.score(....,.....)
print(accuracy)


In [ ]:
## Solution

# Step 1: compute the predictions
predictions_test = rfc.predict(data_test) 

# Step 2: display the confusion matrix

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(target_test, predictions_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=rfc.classes_)
disp.plot() 

# Step 3: compute the accuracy
accuracy = rfc.score(data_test,target_test)
print(accuracy)


In [ ]:
## Solution

# Step 1: compute the predictions
predictions_test = rfc.predict(data_test) 

# Step 2: display the confusion matrix

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(target_test, predictions_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=rfc.classes_)
disp.plot() 

# Step 3: compute the accuracy
accuracy = rfc.score(data_test,target_test)
print(accuracy)


Comment: .........

Number of misclassified observations in the test set:............

Question 3. We will now focus on the calibration of some RF parameters: `n_estimators` and `max_features`. To calibrate these parameters, we will used the OOB errors. The code below shows how the OOB error can be measured at the addition of each new tree during training. The resulting plot can be used to approximate a suitable value of `n_estimators` at which the OOB error stabilizes. Comment this plot. What value for `n_estimators` does it seem suitable ?

*Indication: a suitable value for `n_estimators` is a value for which the oob error of the forest is stable.*


In [ ]:
import matplotlib.pyplot as plt

RANDOM_STATE = 0


# Map a classifier name to a list of (<n_estimators>, <error rate>) pairs.
error_rate = []

# Range of `n_estimators` values to explore.
min_estimators = 100
max_estimators = 800
step=5

for i in range(min_estimators, max_estimators+1, step):
    rf = RandomForestClassifier(warm_start=True, n_estimators=i, max_features='auto',random_state=RANDOM_STATE, oob_score=True)
    rf.fit(data, target)

    # Record the OOB error for each `n_estimators=i` setting.
    oob_error = 1 - rf.oob_score_
    error_rate.append(oob_error)
  

# Plot Generate the "OOB error rate" vs. "n_estimators" plot

plt.plot(range(min_estimators, max_estimators +1, step), error_rate, label="OOB error rate")

plt.ylim(0, 1.5*max(error_rate)) 
plt.xlim(min_estimators, max_estimators)
plt.xlabel("n_estimators")
plt.ylabel("OOB error rate vs. number of trees")
plt.legend(loc="upper right")
plt.show()


In [ ]:
import matplotlib.pyplot as plt

RANDOM_STATE = 0 # Note: if you modifiy the random stat, the results can be sligthly different


# Map a classifier name to a list of (<n_estimators>, <error rate>) pairs.
error_rate = []

# Range of `n_estimators` values to explore.
min_estimators = 100
max_estimators = 800
step=5

for i in range(min_estimators, max_estimators+1, step):
    rf = RandomForestClassifier(warm_start=True, n_estimators=i, max_features='auto',random_state=RANDOM_STATE, oob_score=True)
    rf.fit(data, target)

    # Record the OOB error for each `n_estimators=i` setting.
    oob_error = 1 - rf.oob_score_
    error_rate.append(oob_error)
  

# Plot Generate the "OOB error rate" vs. "n_estimators" plot

plt.plot(range(min_estimators, max_estimators +1, step), error_rate, label="OOB error rate")

plt.ylim(0, 1.5*max(error_rate)) 
plt.xlim(min_estimators, max_estimators)
plt.xlabel("n_estimators")
plt.ylabel("OOB error rate vs. number of trees")
plt.legend(loc="upper right")
plt.show()

Comment of the plot and choice of a suitable value for `n_estimators`: .............






Question 4. Now, we will repeat `B` times a `k`-fold cross validation with the function GridSearchCV to calibrate at the same time the parameters `n_estimators` and `max_features`. Complete the command below, comment the results and select suitable values for the parameters `n_estimators` and `max_features`.

*Indications:*
- *the values considers for `n_estimators` and `max_features` are `max_features = (0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9)` and `n_estimators=(400,600,800)`. The values (0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9) for `max_features` represent the proportions of selected features. See the documention of [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier) class.*
- *we will repeat `B=10`times a k-fold crossvalidation with `k=3`.*

In [ ]:
##------- Complete the command below by filling in the gaps '...'.-------##

# Step 1: create a grid with all the values that we will considers for the two paramters
grid = {
    'max_features':[..........], 
    'n_estimators':[...........]
}

# Step 2: use the grid to to search for the best couple of parameters
from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier() # create the forest model to tune

B=.......
results_cv=pd.DataFrame()
for i in range(B):
    
    rf_cv = GridSearchCV(estimator=rf,param_grid=grid, cv=......,n_jobs=-1)# Search the best values for the parameters using 3-fold cross validation, and use all available cores(n_jobs=-1)
    rf_cv.fit(data, target) # Fit the CV search
    if i==0: 
        results_cv=pd.DataFrame(rf_cv.cv_results_)[["params","mean_test_score","std_test_score"]]
    else:
        results_cv["mean_test_score"]=results_cv["mean_test_score"]+pd.DataFrame(rf_cv.cv_results_)["mean_test_score"]
        results_cv["std_test_score"]=results_cv["std_test_score"]+pd.DataFrame(rf_cv.cv_results_)["std_test_score"]  
        
        

results_cv["mean_test_score"]=results_cv["mean_test_score"]/B
results_cv["std_test_score"]=results_cv["std_test_score"]/B 


# Step 3: get the best parameters (with the higher performance)
ind_best=results_cv["mean_test_score"].idxmax()
print(results_cv["params"].iloc[ind_best])
print(results_cv["mean_test_score"].iloc[ind_best])
print(results_cv["std_test_score"].iloc[ind_best])
print(results_cv)

In [ ]:
##------- Complete the command below by filling in the gaps '...'.-------##

# Step 1: create a grid with all the values that we will considers for the two paramters
grid = {
    'max_features':[..........], 
    'n_estimators':[...........]
}

# Step 2: use the grid to to search for the best couple of parameters
from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier() # create the forest model to tune

B=.......
results_cv=pd.DataFrame()
for i in range(B):
    
    rf_cv = GridSearchCV(estimator=rf,param_grid=grid, cv=......,n_jobs=-1)# Search the best values for the parameters using 3-fold cross validation, and use all available cores(n_jobs=-1)
    rf_cv.fit(data, target) # Fit the CV search
    if i==0: 
        results_cv=pd.DataFrame(rf_cv.cv_results_)[["params","mean_test_score","std_test_score"]]
    else:
        results_cv["mean_test_score"]=results_cv["mean_test_score"]+pd.DataFrame(rf_cv.cv_results_)["mean_test_score"]
        results_cv["std_test_score"]=results_cv["std_test_score"]+pd.DataFrame(rf_cv.cv_results_)["std_test_score"]  
        
        

results_cv["mean_test_score"]=results_cv["mean_test_score"]/B
results_cv["std_test_score"]=results_cv["std_test_score"]/B 


# Step 3: get the best parameters (with the higher performance)
ind_best=results_cv["mean_test_score"].idxmax()
print(results_cv["params"].iloc[ind_best])
print(results_cv["mean_test_score"].iloc[ind_best])
print(results_cv["std_test_score"].iloc[ind_best])
print(results_cv)

Result interpretation and choice of tuned values for `n_estimators` and `max_features`: ........

Solution for the interpretation and the selection of values for `n_estimators` and `max_features`: .......

Question 5. Build the random forest `opt_rfc` by using the best values for the parameters `n_estimators` and `max_features`. Compute the accuracy on the test set and display the confusion matrix. 

In [ ]:
##------- Complete the command below by filling in the gaps '...'.-------##

# Step 1: fit a random forest with the best values for parameters
opt_rfc = RandomForestClassifier(n_estimators=...., max_features=....,oob_score=True,random_state=0)

# Step 2: build the random forest on the train set by indicating the input data and the target variable 
opt_rfc.fit(...,.....)

# Step 3: compute the accuracy and the confusion matrix
predictions_test_2 = opt_rfc.predict(....) 
cm_2 = confusion_matrix(...., .....)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_2,display_labels=opt_rfc.classes_)
disp.plot() 

# Step 4: compute the accuracy
accuracy_2 = opt_rfc.score(....,.....)
print(accuracy_2)

Question 6. what value for`max_features` we have to choose if we want to apply the bagging algorithm with CART instead of a random forest ? Build this model and compute the prediction error of this model based on the test set. We will call this model $bag$.

*Indication: use the value selected at `question 5`for the parameter `n_estimators`.* 

In [ ]:
##------- Complete the command below by filling in the gaps '...'.-------##

# Step 1: fit a a bagging model (use the best value for n_estimators)
bag = RandomForestClassifier(n_estimators=...., max_features=....,oob_score=True,random_state=0)

# Step 2: build the random forest on the train set by indicating the input data and the target variable 
bag.fit(...., ......)

# Step 3: compute the prediction on the test set and the confusion matrix
predictions_test_bag = bag.predict(......) 
cm_bag = confusion_matrix(.....,......)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_bag,display_labels=bag.classes_)
disp.plot() 

# Step 4: compute the accuracy on the test set
accuracy_bag = bag.score(....,.....)
print(accuracy_bag)



Question 8. Compare your three models `opt_rfc`, `bag`and `rfc` using suitable performance criteria. What model do you choose and why ?

Your Answer: ........




Question 9. The code below shows the boxplot of the mean decrease in accuracy (MDA) for the 15 features with the highest average MDA. The MDA of each of the 147 features has been independently computed `n__repeats =10` times on the test set for the random forest `opt_rfc` (at each run, another permutation is applied for the features). Comment the plot. 

In [ ]:
from sklearn.inspection import permutation_importance
result = permutation_importance(opt_rfc, data_test, target_test, n_repeats=10, random_state=0, n_jobs=2)

sorted_idx = result.importances_mean.argsort()
invert_sorted_idx=sorted_idx[::-1][:14]# keep only the 15 features with the highest average MDA

fig, ax = plt.subplots()
ax.boxplot(result.importances[invert_sorted_idx].T,
           vert=False, labels=data_test.columns[invert_sorted_idx])
ax.set_title("MDA computed on the test set for the 15 features with the highest average MDA (average MDA is displayed in orange")
fig.tight_layout()
plt.show()

Comments about the plot above: .......

## Ex 2: introduction to regression trees

Here, we consider the dataset named *srbct_data*. It is relative to small round blue cell tumors of childhood. This set is composed of :

- a response factor of length 63, called class, indicating the class of each sample (4 classes in total).
- 2308 predictors. Each predictor represents the expression of one gene. The features are correlated. 

More information about the data are available on https://www.rdocumentation.org/packages/plsgenomics/versions/1.5-2/topics/SRBCT 

The table named *genes_name* contains the names of the genes and a description for each gene.

In [ ]:
# Load the data 
import pandas as pd # data analysis
srbct_data = pd.read_csv("cancer_data.csv")
genes_name = pd.read_csv("cancer_data_genes_names.csv")

In [ ]:
# Have a look at the data 
srbct_data.head()

In [ ]:
genes_name.head()

In [ ]:
# Display the dimension and a brief statistics summary
srbct_data.shape

In [ ]:
# Display a brief statistics summary
srbct_data.describe(include='all')

In [ ]:
## Display a frequency table for the target variable named `class`.
srbct_data["class"].value_counts()

In [ ]:
# Select the variables that we be used 
target_name = "class" # The response variable that we will consider
features_names = list(srbct_data)
features_names.remove('class') # The predictors/features used to predict the target
#print(target_column)
#print(features_columns)

In [ ]:
# Split the dataset into train (75% of data) and test dataset (25% of data)
from sklearn.model_selection import train_test_split
X, y =  srbct_data[features_names], srbct_data[target_name]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0,test_size=0.25)
print(X_train.shape)
print(X_test.shape)

**Questions:**
 - 1) Build a random forest on this dataset by using the [RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor) class. Read carefully the documentation and use the default values for the RF parameters.
 - 2) Calibrate the two parameters `n_estimators` and `max_features` by using the same approach as in `Ex1 question 4`. Then, select values for `n_estimators` and `max_features`.
 - 3) Build a second random forest using the selected values for `n_estimators` and `max_features`.
 - 4) Because there are lots of features and they are correlated, use the MDA score to select a subset of only 20 variables (use the same approach as in `Ex1 question 9`). Justify your choice.
 - 5) Build a third random forest based only the selected subset of features.
 - 6) Compare the three models. What model do you select and why ?

In [ ]:
##---- Write your answer ----##






